<a href="https://colab.research.google.com/github/DrHeogg/Customer-data-parser/blob/main/%D0%9F%D0%9F%D0%9F3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Подгружаем нужные либы**

In [ ]:
# Автоматическая установка всех зависимостей
!pip install -q natasha pymorphy2 spacy openpyxl ipywidgets
!python -m spacy download ru_core_news_sm

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 68.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'R

# **Главная часть - экстрактор данных**

In [ ]:


import pandas as pd
import re
import pymorphy2
import openpyxl
import spacy
from io import BytesIO
from google.colab import files
from natasha import (
    Segmenter,
    MorphVocab,
    NewsNERTagger,
    NamesExtractor,
    NewsEmbedding,
    Doc
)
from natasha.extractors import AddrExtractor
from ipywidgets import widgets, VBox, HBox, Layout
from IPython.display import display, clear_output

# Инициализация сегментаторов, экстракторов и т.д.
emb = NewsEmbedding()
segmenter = Segmenter()
morph_vocab = MorphVocab()
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)
address_extractor = AddrExtractor(morph_vocab)
morph = pymorphy2.MorphAnalyzer()
nlp = spacy.load("ru_core_news_sm")


def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

def lemmatize_word(word):
    parsed_word = morph.parse(word)[0]
    return parsed_word.normal_form

def find_emails(text):
    email_pattern = re.compile(r"""
        [a-zA-Z0-9._%+-]+
        @
        [a-zA-Z0-9.-]+
        \.
        [a-zA-Z]{2,}
    """, re.VERBOSE)
    emails = email_pattern.findall(text)
    filtered_emails = [email.strip() for email in emails if '@' in email and '.' in email]
    return emails

def find_general_emails(text):
    general_email_pattern = re.compile(r"""
        (?:info|manager|secretary|mail)
        @
        [\w.-]+
        \.
        [a-z]{2,}
    """, re.VERBOSE)
    general_emails = general_email_pattern.findall(text)
    filtered_general_emails = [email.strip() for email in general_emails if '@' in email and '.' in email]
    return general_emails

def process_df(df, positions_dict, organization_column):

    df.rename(columns={organization_column: 'Организация'}, inplace=True)
    df['Телефоны'] = ''
    df['Имя'] = ''
    df['Фамилия'] = ''
    df['Отчество'] = ''
    df['Почты'] = ''
    df['Общие почты'] = ''
    df['Должность'] = ''
    df['Адрес'] = ''
    df['Город'] = ''


    phone_pattern = re.compile(r"""
        (\(?\d{3,4}\)?[\s-]?\d{3}[\s-]?\d{2}[\s-]?\d{2})
        |
        (\(?\d{3,4}\)?[\s-]?\d{3}[\s-]?\d{2}[\s-]?\d{2}\s*[,\.]\sдоб\.\s\d{3,4})
        |
        (\(\d{3,4}\)\s*\d{2}-\d{2}-\d{2})
    """, re.VERBOSE)

    for index, row in df.iterrows():
        emails = []
        phones = set()
        general_emails = []
        names = []
        surnames = []
        patronymics = []
        positions = []
        addresses = []
        cities = []

        for cell in row:
            cell_str = str(cell)
            emails.extend(find_emails(cell_str))
            phones.update(phone_pattern.findall(cell_str))
            general_emails.extend(find_general_emails(cell_str))


            doc = Doc(cell_str)
            doc.segment(segmenter)
            doc.tag_ner(ner_tagger)

            for span in doc.spans:
                span.normalize(morph_vocab)

                if span.type == 'PER':
                    span.extract_fact(names_extractor)
                    if span.fact:
                        fact_dict = span.fact.as_dict
                        if 'first' in fact_dict:
                            names.append(fact_dict['first'])
                        if 'last' in fact_dict:
                            surnames.append(fact_dict['last'])
                        if 'middle' in fact_dict:
                            patronymics.append(fact_dict['middle'])


            spacy_doc = nlp(cell_str)
            for ent in spacy_doc.ents:
                if ent.label_ == "LOC":
                    addresses.append(ent.text)


            for token in spacy_doc:
                if token.text:
                    cleaned_token = remove_punctuation(token.text)
                    lemma = lemmatize_word(cleaned_token)
                    if lemma.lower() in positions_dict:
                        positions.append(lemma)


        emails = [email for email in emails if email not in general_emails]

        df.at[index, 'Почты'] = ', '.join(emails)
        df.at[index, 'Телефоны'] = ', '.join([''.join(num) for num in phones])
        df.at[index, 'Имя'] = ', '.join(names)
        df.at[index, 'Фамилия'] = ', '.join(surnames)
        df.at[index, 'Отчество'] = ', '.join(patronymics)
        df.at[index, 'Общие почты'] = ', '.join(general_emails)
        df.at[index, 'Должность'] = ', '.join(positions)
        df.at[index, 'Адрес'] = ', '.join(addresses)

    return df[['Имя', 'Фамилия', 'Отчество', 'Телефоны', 'Почты', 'Общие почты', 'Должность', 'Адрес', 'Организация']]

positions_dict = {
    "инженер", "отдел", "комплектация", "начальник", "ЭМС", "заведующий", "лаборатория", "главный", "конструктор",
    "заместитель", "директор", "проект", "развитие", "коммерческий", "деятельность", "генеральный", "директор",
    "КТехнология", "продажа", "центр", "технологический", "проект", "специалист", "работа", "персонал", "менеджер",
    "консультант", "аналитик", "руководитель", "по", "для", "в", "на", "от", "с", "гл", "ген", "зам", "нач", "инж", "дир",
    "спец", "конс", "ана", "рук", "мнж", "отд", "лаб", "проект", "деят", "компл", "техн", "перс", "раб", "прод",
    "комм", "разв", "ЭМ", "КТ", "маркетолог", "НИО", "разработка", "аппаратный", "обеспечение", "наука",
    "НИ", "разработка", "аппарат", "обеспечение", "наука", "ведущий", "специалист", "отдел", "оценка", "качество", "технический",
    "программист", "тестировщик", "разработчик", "системный", "администратор", "аналитик", "данные", "архитектор", "система",
    "бизнес", "аналитик", "финансовый", "директор", "финансовый", "менеджер", "бухгалтер", "юрист", "правовой", "отдел",
    "экономист", "кадровик", "кадровый", "специалист", "логист", "закупка", "менеджер", "поставка", "склад", "хранение",
    "HR", "менеджер", "рекрутер", "маркетолог", "SMM", "специалист", "PR", "менеджер", "контент", "менеджер", "продажа",
    "дизайнер", "web", "разработчик", "frontend", "backend", "mobile", "devops", "инженер", "QA", "тестировщик",
    "продукт", "менеджер", "бизнес", "аналитик", "проектный", "менеджер", "продажа", "менеджер", "продажа", "представитель",
    "менеджер", "клиентский", "служба", "секретарь", "управляющий", "директор", "генеральный", "директор", "основатель",
    "владелец", "руководитель", "отдел", "поддержка", "помощник", "руководитель", "секретарь", "управляющий", "директор",
    "преподаватель", "тьютор", "лектор", "коуч", "консультант", "эксперт", "аналитик", "исследователь", "исследователь",
    "методист", "ведущий", "специалист", "методист", "преподаватель", "тьютор", "ассистент", "преподаватель", "старший",
    "преподаватель", "профессор", "доцент",
    # Дополнительные термины
    "технический директор", "техдиректор", "техдир", "технолог", "проектировщик", "архитектор", "база данных", "data scientist",
    "data engineer", "data analyst", "machine learning engineer", "AI specialist", "робототехник", "механик", "электрик",
    "электронщик", "радиоинженер", "системный аналитик", "сисадмин", "сетевой инженер", "сетевой администратор", "IT специалист",
    "IT менеджер", "IT директор", "IT специалист", "IT аналитик", "IT консультант", "IT эксперт", "IT специалист", "IT разработчик",
    "IT тестировщик", "IT техподдержка", "IT администратор", "IT менеджер", "IT директор"
}


upload_button = widgets.FileUpload(
    accept='.xlsx, .xls',
    multiple=False,
    description='Загрузить основной файл',
    layout=widgets.Layout(width='300px', height='50px')
)

merge_checkbox = widgets.Checkbox(
    value=False,
    description='Объединить с другим файлом',
    layout=widgets.Layout(width='300px', height='50px')
)

extract_button = widgets.Button(
    description='Извлечь',
    button_style='success',
    layout=widgets.Layout(width='300px', height='50px')
)

output = widgets.Output()
merge_output = widgets.Output()


df = None
df_processed = None
df_merge = None


column_dropdown = widgets.Dropdown(
    options=[],
    description='Колонка компаний:',
    disabled=False,
    layout=widgets.Layout(width='300px', height='50px')
)


merge_upload_button = widgets.FileUpload(
    accept='.xlsx, .xls',
    multiple=False,
    description='Загрузить файл для объединения',
    layout=widgets.Layout(width='300px', height='50px')
)
merge_upload_button.layout.display = 'none'

def on_merge_checkbox_change(change):
    if merge_checkbox.value:

        merge_upload_button.layout.display = 'block'
    else:

        merge_upload_button.layout.display = 'none'

def on_upload_change(change):
    if upload_button.value:
        uploaded_filename = list(upload_button.value.keys())[0]
        content = upload_button.value[uploaded_filename]['content']
        try:
            global df
            df = pd.read_excel(BytesIO(content))
            column_dropdown.options = df.columns.tolist()
            with output:
                clear_output()
                display(VBox([column_dropdown, merge_checkbox, merge_upload_button, extract_button]))
        except Exception as e:
            with output:
                clear_output()
                print(f"Ошибка при чтении файла: {e}")

def on_extract_button_clicked(b):
    selected_column = column_dropdown.value
    if not selected_column:
        with output:
            clear_output()
            print("Пожалуйста, выберите колонку с названием компаний.")
        return
    global df_processed
    df_processed = process_df(df, positions_dict, selected_column)
    if merge_checkbox.value:
        if merge_upload_button.value:
            uploaded_filename = list(merge_upload_button.value.keys())[0]
            content = merge_upload_button.value[uploaded_filename]['content']
            try:
                global df_merge
                df_merge = pd.read_excel(BytesIO(content))
                df_final = pd.concat([df_processed, df_merge], ignore_index=True)
                df_final.to_excel('merged_output.xlsx', index=False)
                with output:
                    clear_output()
                    print("Объединение завершено. Вы можете скачать файл.")
                    files.download('merged_output.xlsx')
            except Exception as e:
                with output:
                    clear_output()
                    print(f"Ошибка при чтении файла для объединения: {e}")
        else:
            with output:
                clear_output()
                print("Пожалуйста, загрузите файл для объединения.")
    else:
        df_processed.to_excel('processed_output.xlsx', index=False)
        with output:
            clear_output()
            print("Обработка завершена. Вы можете скачать файл.")
            files.download('processed_output.xlsx')

upload_button.observe(on_upload_change, names='value')
merge_checkbox.observe(on_merge_checkbox_change, names='value')
extract_button.on_click(on_extract_button_clicked)

display(VBox([upload_button, output]))

# **Обогащение данными по рассылкам**

In [ ]:

import pandas as pd
from io import BytesIO
from google.colab import files
from ipywidgets import widgets, VBox, Layout
from IPython.display import display, clear_output

upload_data_button = widgets.FileUpload(
    accept='.xlsx, .xls',
    multiple=False,
    description='Загрузить файл с данными',
    layout=widgets.Layout(width='300px', height='50px')
)
upload_report_button = widgets.FileUpload(
    accept='.xlsx, .xls',
    multiple=False,
    description='Загрузить отчёт по рассылке',
    layout=widgets.Layout(width='300px', height='50px')
)
process_button = widgets.Button(
    description='Проверить совпадения',
    button_style='success',
    layout=widgets.Layout(width='300px', height='50px')
)
output = widgets.Output()

data_df = None
report_df = None

def load_data(change):
    """Загрузка файла с основными данными"""
    if upload_data_button.value:
        file_name = list(upload_data_button.value.keys())[0]
        content = upload_data_button.value[file_name]['content']
        global data_df
        data_df = pd.read_excel(BytesIO(content))
        with output:
            clear_output()
            print(f"Файл с данными '{file_name}' успешно загружен.")

def load_report(change):
    """Загрузка отчёта по рассылке"""
    if upload_report_button.value:
        file_name = list(upload_report_button.value.keys())[0]
        content = upload_report_button.value[file_name]['content']
        global report_df
        report_df = pd.read_excel(BytesIO(content), sheet_name=None)
        with output:
            clear_output()
            print(f"Отчёт по рассылке '{file_name}' успешно загружен.")

def process_data(b):
    """Поиск совпадений и добавление колонок с результатами рассылки"""
    if data_df is None or report_df is None:
        with output:
            clear_output()
            print("Пожалуйста, загрузите оба файла перед проверкой.")
        return

    data_df['Результат отправки'] = 'unknown'
    data_df['Время обновления'] = 'unknown'

    report_values = pd.concat(
        [df.iloc[:, 0] for df in report_df.values()]
    ).astype(str).unique()

    for index, row in data_df.iterrows():
        общие_почты = str(row.get('Общие почты', ''))
        почты = str(row.get('Почты', ''))

        if общие_почты in report_values or почты in report_values:
            for sheet_name, df in report_df.items():
                match = df[df.iloc[:, 0].astype(str).isin([общие_почты, почты])]
                if not match.empty:
                    data_df.at[index, 'Результат отправки'] = match.iloc[0, 1]
                    data_df.at[index, 'Время обновления'] = match.iloc[0, 2]
                    break

    data_df.to_excel('updated_data.xlsx', index=False)
    with output:
        clear_output()
        print("Обработка завершена. Вы можете скачать обновлённый файл.")
        files.download('updated_data.xlsx')

upload_data_button.observe(load_data, names='value')
upload_report_button.observe(load_report, names='value')
process_button.on_click(process_data)

display(VBox([
    upload_data_button,
    upload_report_button,
    process_button,
    output
]))


# **Склеивание 2 файлов, на всякий случай**

In [ ]:

import pandas as pd
from io import BytesIO
from google.colab import files
from ipywidgets import widgets, VBox, Layout
from IPython.display import display, clear_output

upload_file1_button = widgets.FileUpload(
    accept='.xlsx, .xls',
    multiple=False,
    description='Загрузить файл 1',
    layout=widgets.Layout(width='300px', height='50px')
)
upload_file2_button = widgets.FileUpload(
    accept='.xlsx, .xls',
    multiple=False,
    description='Загрузить файл 2',
    layout=widgets.Layout(width='300px', height='50px')
)
merge_button = widgets.Button(
    description='Склеить файлы',
    button_style='success',
    layout=widgets.Layout(width='300px', height='50px')
)
output = widgets.Output()

df1 = None
df2 = None

def load_file1(change):
    """Загрузка первого файла"""
    if upload_file1_button.value:
        file_name = list(upload_file1_button.value.keys())[0]
        content = upload_file1_button.value[file_name]['content']
        global df1
        df1 = pd.read_excel(BytesIO(content))
        with output:
            clear_output()
            print(f"Файл 1 '{file_name}' успешно загружен.")

def load_file2(change):
    """Загрузка второго файла"""
    if upload_file2_button.value:
        file_name = list(upload_file2_button.value.keys())[0]
        content = upload_file2_button.value[file_name]['content']
        global df2
        df2 = pd.read_excel(BytesIO(content))
        with output:
            clear_output()
            print(f"Файл 2 '{file_name}' успешно загружен.")

def merge_files(b):
    """Склейка двух файлов и сохранение результата"""
    if df1 is None or df2 is None:
        with output:
            clear_output()
            print("Пожалуйста, загрузите оба файла перед склейкой.")
        return

    try:
        merged_df = pd.concat([df1, df2], ignore_index=True)
        merged_df.to_excel('merged_output.xlsx', index=False)
        with output:
            clear_output()
            print("Файлы успешно склеены. Вы можете скачать результат.")
            files.download('merged_output.xlsx')
    except Exception as e:
        with output:
            clear_output()
            print(f"Ошибка при склейке файлов: {e}")

upload_file1_button.observe(load_file1, names='value')
upload_file2_button.observe(load_file2, names='value')
merge_button.on_click(merge_files)

display(VBox([
    upload_file1_button,
    upload_file2_button,
    merge_button,
    output
]))
